In [12]:
# 218. The Skyline Problem
class Solution(object):
    def getSkyline(self, blds):
        """
        :type blds: List[List[int]]
        :rtype: List[List[int]]
        """
        # blds: [Li, Ri, Hi]
        if not blds:
            return []
        if len(blds) == 1:
            return [[blds[0][0],blds[0][2]], [blds[0][1],0]]
        mid = len(blds) // 2
        left = self.getSkyline(blds[:mid])
        right = self.getSkyline(blds[mid:])
        return self.merge(left, right)
    
    def merge(self, left, right):
        maxH = leftH = rightH = 0
        pl = pr = 0
        nl = len(left)
        nr = len(right)
        res = []
        while pl < nl and pr < nr:
            if left[pl][0] < right[pr][0]:
                pos, leftH = left[pl] # returns bottom left
                pl += 1
            elif left[pl][0] > right[pr][0]:
                pos, rightH = right[pr]
                pr += 1
            else:
                pos, leftH = left[pl]
                rightH = right[pr][1]
                pl += 1
                pr += 1
            maxH = max(leftH, rightH)
            # update result
            if not res or maxH != res[-1][1]:
                res += [[pos, maxH]]
        if pl < nl:
            res += left[pl:]
        if pr < nr:
            res += right[pr:]
        return res
        
blds = [[2,9,10],[3,7,15],[5,12,12],[15,20,10],[19,24,8]]
Solution().getSkyline(blds)

[[2, 10], [3, 15], [7, 12], [12, 0], [15, 10], [20, 8], [24, 0]]

In [11]:
# 218. The Skyline Problem
class Solution(object):
    def getSkyline(self, blds):
        """
        :type blds: List[List[int]]
        :rtype: List[List[int]]
        """
        # blds: [Li, Ri, Hi]
        if not blds: return []
        if len(blds) == 1: return [[blds[0][0], blds[0][2]], [blds[0][1], 0]]
        mid = len(blds) // 2
        left = self.getSkyline(blds[:mid])
        right = self.getSkyline(blds[mid:])
        return self.merge(left, right)
    
    def merge(self, left, right):
        h1, h2, res = 0, 0, []
        while left and right:
            if left[0][0] < right[0][0]:
                pos, h1 = left[0]
                left = left[1:]
            elif left[0][0] > right[0][0]:
                pos, h2 = right[0]
                right = right[1:]
            else:
                pos, h1 = left[0]
                h2 = right[0][1]
                left = left[1:]
                right = right[1:]
            H = max(h1, h2)
            if not res or H != res[-1][1]:
                res.append([pos, H])
        if left:
            res += left
        if right:
            res += right
        return res
    
# buildings = [[2,9,10],[3,7,15]]
buildings = [[2,9,10],[3,7,15],[5,12,12],[15,20,10],[19,24,8]]
Solution().getSkyline(buildings)

[[2, 10], [3, 15], [7, 12], [12, 0], [15, 10], [20, 8], [24, 0]]

In [ ]:
from heapq import *

class Solution:
    def getSkyline(self, LRH):
        skyline = []
        i, n = 0, len(LRH)
        liveHR = []
        while i < n or liveHR:
            if not liveHR or i < n and LRH[i][0] <= -liveHR[0][1]:
                x = LRH[i][0]
                while i < n and LRH[i][0] == x:
                    heappush(liveHR, (-LRH[i][2], -LRH[i][1]))
                    i += 1
            else:
                x = -liveHR[0][1]
                while liveHR and -liveHR[0][1] <= x:
                    heappop(liveHR)
            height = len(liveHR) and -liveHR[0][0]
            if not skyline or height != skyline[-1][1]:
                skyline += [x, height],
        return skyline
题目要找天际线的关键点，首先从左至右把可能出现关键点的位置扫描一遍，
然后判断每个位置是否有关键点，并且确定关键点的具体坐标（也就是其高度）。
通过画图画出大部分可能的重叠情况，我们可以发现，所有的关键点可以分为两类：
进入点和离开点。对于进入点来说，如果此刻进入的这栋楼是最高的，
那他的起点就是一个关键点；如果他不是最高的，或者跟当前最高的楼一样高，
说明该进入点被覆盖了，不需要添加关键点。
对于离开点来说，如果他是最高的楼，那他的离开，一定会导致原本比他更低一点的楼露出，
因此对应的关键点高度就是比他更低一点的楼的高度；如果他不是最高的，
那他的离开就对天际线没有影响，因此不需要更新。
在代码实现时，首先我们需要把“进入点”和“离开点”都加入到一个队列中，
并按照从左往右的顺序开始排序。这里的构思十分巧妙。首先考虑进入点，
对于每栋楼的坐标(L,R,H)来说，我们先按照左下标来排序，即key为L。
但是当两栋楼的左下标一样时，我们应该优先考虑更高的那栋楼。
因此key为(L,-H)。然后再考虑离开点，离开点的右下标可以同样作为L与进入点一起排序，
设想一种边界情况：当某个位置同时出现多栋楼的离开点和进入点时，
此处是否有关键点以及相应关键点的高度取决于此时仍然存在的楼的高度，
以及刚好出现的楼的高度，也就是说，进入点必须比离开点先考虑，
而离开点之间的先后顺序并无影响，因为每栋楼的离开点所在的关键点不会由这栋楼的高度影响，
另外一种情况是，如果某个位置只有离开点，没有进入点，且没有任何还存在的楼，
那(R,0)也需要加入的集合里。为了区分进入点和离开点，我们把进入点表达为(L,-H,R)，
把离开点表达为(R,0,None)，因此只有进入点的元组中有三个存在的数值。
然后按照tuple[0]和tuple[1]来排序，以保证上述边界情况下，
考虑点的优先顺序都是正确的。
接下来就是要遍历排序后的所有候选点。首先我们需要一个有序的数据结构，
能够维护当前最高的一栋楼的高度和位置信息，这里的构思也非常巧妙。
为什么需要知道当前最高楼的高度？因为天际线一定是当前所有楼层最高的那条，
如果在遍历过程中，当前的天际线与目前最高楼的高度不一样，说明发生了两种情况之一：
一种是出现了一栋更高的楼，此时一定是经过了一个新的进入点，
所以此时要加入一个新的天际线关键点，其高度就是当前最高楼的高度；另一种情况是，
最高楼的高度变低了，说明一定是经过了一个离开点，有一栋最高楼在刚才离开了，
天际线发生了变化，因此也要加入一个新的天际线关键点，其高度就是刚才最高楼离开之后，
剩下楼层里最高的高度。如果目前已经没有任何大楼，说明到达了最低点。
因此我们在初始化这个有序的结构时，需要加一个高度为0，横坐标位置无限大的点，
用来完成这种“特殊情况”。
总结来说，就是只要当前维护的最大楼高度不等于上一个添加的天际线关键点的高度，
就需要增加一个新的关键点了。另外一个边界情况是，当目前还没有任何天际线时，
我们如何判断当前最大楼高是否发生了变化呢？也就是在我们遍历第一栋楼时，
我们应该直接加入这个进入点，因为这个进入点一定高于地平线，
地平线就是上一个“天际线”。所以我们需要在初始化最终集合的时候，
加入一个高度为0的元素，来解决这个问题。当然在最后返回的时候，
还需要扔掉这个元素。除此之外也可以通过增加if逻辑来应对这种情况。
最后一个问题是，我们用什么数据结构来维护当前的最高楼层，
以及如何在遍历过程中进行更新。首先我们思考一下，需要进行哪些操作：
取出当前最大的楼层高度；删除已经离开的楼层高度；增加刚进入的楼层高度。
何时需要增加楼层高度？当此时的点是进入点时。何时需要删除已经离开点楼层高度？
当此时遍历的点已经超过了楼层的最右位置时。也就是说，我们需要存的数据，
应该包含了楼的高度和最右位置。总结一下就是：每次当我们遍历到进入点时，
把对应的楼高和右下标加入到这个有序结构中；
每次当我们遍历到的点的位置已经超过或者等于当前最高楼的右下标时，
说明这栋楼已经失效（为什么等于也不行：因为最开始分析过，
楼的离开对这栋楼所构成的天际线是没有影响的，可以看作是左闭右开的区间），
这个操作需要一直执行，直到当前的最高楼仍然在范围内为止。总结到这里，
我们就可以开始选择合适的数据结构。
如何能够快速的维护最大值、删除最大值以及增加一个新的元素？
答案就是堆。python中heapq库可以方便地维护一个有序的list，
通过heappop和heappush可以方便地增删元素。
取最大值则只需要通过获取第一个元素（注意heapq默认是升序排列，
因此我们仍存储-H来实现降序）。